In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display

In [2]:
df = pd.read_csv("credit_risk_dataset – clean.csv")
print(df.head())

# Helper function:
# === For Quick Visual Check (numbers) ==========================================================
def audit(column_name):
    """
    Performs a basic statistical audit of a numerical column.
    Displays descriptive statistics (describe), extreme values (TOP/BOTTOM 5), 
    and calculates the count and percentage of missing values (NaN).
    """
    
    # Check for error    
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the dataframe.")
        return

    print(f"---  AUDIT: {column_name.upper()} ---")
    print(df[column_name].describe())

    print(f"\nBOTTOM 5 ({column_name}):")
    print(df[column_name].nsmallest(5))

    print(f"\nTOP 5 ({column_name}):")
    print(df[column_name].nlargest(5))

    null_count = df[column_name].isnull().sum()
    null_pct = (null_count / len(df)) * 100
    
    print(f"\nMissing values: {null_count} ({null_pct:.2f} %)")
    print("-" * 30)

# === For capping outliers + boxplot ===========================================================
def cap_outliers_iqr(column_name):
    """
    Creates a boxplot and cap outliers from the specified attribute.
    The cleaned data is stored in a new column named {column_name}_cleaned.
    Displays a summary of the changes performed.
    """
    # Check for error
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found.")
        return df

    # Create boxplot
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=df[column_name], color="skyblue")
    plt.title(f"Boxplot: {column_name}")
    plt.xlabel("Values")
    plt.show()

    # IQR quartile calculation 
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Create new column
    new_col_name = f"{column_name}_cleaned"
    
    # Check for same existing column
    if new_col_name in df.columns:
        df.drop(columns=[new_col_name], inplace=True)

    # Capping
    capped_series = df[column_name].clip(lower=lower_bound, upper=upper_bound)

    # Column insertion
    idx = df.columns.get_loc(column_name)
    df.insert(loc=idx + 1, column=new_col_name, value=capped_series)

    # Audit calculations
    num_capped_lower = (df[column_name] < lower_bound).sum()
    num_capped_upper = (df[column_name] > upper_bound).sum()
    total_capped = num_capped_lower + num_capped_upper
    
    print(f"\nAUDIT: {column_name.upper()}")
    print(f"Boxplot limits: Low = {lower_bound:.2f}, High = {upper_bound:.2f}")
    
    print(f"\n--- Statistics (Original vs Cleaned) ---")
    stats_comparison = pd.concat([df[column_name].describe(), df[new_col_name].describe()], axis=1)
    print(stats_comparison)

    print(f"\nValues capped (outside whiskers): {total_capped}")
    print(f" - Capped below lower bound: {num_capped_lower}")
    print(f" - Capped above upper bound: {num_capped_upper}")
    print("-" * 40)
    
    return df

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1673359562.py, line 15)

## Duplicates Check

Checking for duplicates. This dataset does not contain any unique keys, which presents two options: assume that identical rows are duplicates, or retain them because we cannot determine if they are actual duplicates or simply identical values for two distinct records. In this case, I will apply the second option (keeping them), as I lack a clear indication of whether they are duplicates. Furthermore, the number of such occurrences is negligible.

In [ ]:
duplicates = df[df.duplicated()]
print(f"Number of duplicates: {len(duplicates)}")

duplicate_pairs = df[df.duplicated(keep=False)].sort_values(by=["person_income", "person_age"])
display(duplicate_pairs.head(10))

## person_age Cleaning

Since the data contains exaggerated values, I am capping age at 100. The dataset predominantly features younger individuals. While I could address outliers, they won't interfere with this specific analysis primarly focusing on the 25–30 age group. Therefore, I am only removing clear data entry errors

In [ ]:
# Clean person_age + add column person_age_cleaned
idx_age = df.columns.get_loc("person_age")
cleaned_data = df["person_age"].mask(df["person_age"] > 100)

# Error prevention 
if "person_age_cleaned" in df.columns:
    df.drop(columns=["person_age_cleaned"], inplace=True)

df.insert(loc=idx_age + 1, column="person_age_cleaned", value=cleaned_data)

# Control test
audit("person_age")
audit("person_age_cleaned")

## person_emp_length Validation

For `person_emp_length` validation, I'm implementing a simple check: `person_emp_length` > (`person_age` - 15). This assumes a legal minimum working age of 15. Any records exceeding this threshold are considered invalid and removed.

In [ ]:
# Validate legal working age + add column person_emp_length_cleaned
idx = df.columns.get_loc("person_emp_length")
employment_condition = df["person_emp_length"] > (df["person_age"] - 15)
new_emp_col = df["person_emp_length"].mask(employment_condition)

# Error prevention 
if "person_emp_length_cleaned" in df.columns:
    df.drop(columns=["person_emp_length_cleaned"], inplace=True)

df.insert(loc=idx + 1, column="person_emp_length_cleaned", value=new_emp_col)
invalid_count = df["person_emp_length_cleaned"].isna().sum() - df["person_emp_length"].isna().sum()
print(f"Number of records nullified due to logic check: {invalid_count}")

# Control test
audit("person_emp_length")
audit("person_emp_length_cleaned")

## loan_percent_income Validation

In this section, I am validating the `loan_percent_income` field. First, I check for existing errors by comparing the current values with a manual calculation of `loan_amnt` / `person_income` to assess the extent of the discrepancies. I noticed that some errors stem from insufficient decimal precision (rounding to 2 decimal places or 0.00). To improve accuracy, I am increasing the precision to 4 decimal places and recalculating the values accordingly.

In [ ]:
# loan_percent_income - old data validation
expected_pct_2 = (df["loan_amnt"] / df["person_income"]).round(2)
errors_mask_2 = (df["loan_percent_income"].round(2) != expected_pct_2)
print(f"Number of calculation inconsistencies (at 2 decimal places): {errors_mask_2.sum()}")

# loan_percent_income - new re-calculation
expected_pct_4 = (df["loan_amnt"] / df["person_income"]).round(4)
new_col_values = df["loan_percent_income"].copy()
errors_mask_4 = (df["loan_percent_income"].round(4) != expected_pct_4)
new_col_values[errors_mask_4] = expected_pct_4[errors_mask_4]
idx = df.columns.get_loc("loan_percent_income")

# Error prevention 
if "loan_percent_income_cleaned" in df.columns:
    df.drop(columns=["loan_percent_income_cleaned"], inplace=True)

df.insert(loc=idx + 1, column="loan_percent_income_cleaned", value=new_col_values)

print(f"Successfully created "loan_percent_income_cleaned" with 4-decimal precision.")

audit("loan_percent_income")
audit("loan_percent_income_cleaned")

## cb_person_cred_hist_length Flagging

In this step, I am flagging cases where the credit history starts before the age of 15. I've set this as an auxiliary threshold to assess the quality of the dataset and understand the distribution of these potentially problematic records

In [ ]:
# Flags suspicious records
invalid_logic = df[(df["person_age"] - df["cb_person_cred_hist_length"]) < 15]
print(f"Number of inconsistent records identified: {len(invalid_logic)}")

## person_income Outliers

In this step, I am capping outliers in person_income using a 'Winsorization' approach based on the Interquartile Range (IQR). As shown in the initial boxplot, the presence of an extreme outlier (6,000,000) distorts the visualization to the point where the boxplot becomes unreadable. To ensure a more robust analysis and better data distribution, it is necessary to cap these extreme values at the whiskers of the boxplot.

In [ ]:
cap_outliers_iqr("person_income")

## loan_amnt Outliers

Applying the same IQR-based Winsorization logic to `loan_amnt` to handle outliers and maintain consistency across features.

In [ ]:
cap_outliers_iqr("loan_amnt")

In [ ]:
#Save data
df.to_csv("data_cleaned.csv", index=False)